In [3]:
from indexify import IndexifyClient, ExtractionGraph
client = IndexifyClient()

In [ ]:
extraction_graph_spec = """
name: 'testing'
extraction_policies:
   - extractor: 'tensorlake/chunk-extractor'
     name: 'chunker'
     input_params:
        chunk_size: 1000
        overlap: 100
   - extractor: 'tensorlake/minilm-l6'
     name: 'embeddings'
     content_source: 'chunker'
"""

extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
client.create_extraction_graph(extraction_graph)                                            

In [ ]:
from langchain.document_loaders import PyPDFLoader

doc_list = ["sample.pdf"]

for doc_name in doc_list:
#loader = PyPDFLoader("notes_chapter_Introduction.pdf"), 
    pdf_loader = PyPDFLoader(file_path=doc_name)
    documents = pdf_loader.load()

#data = loader.load()

#print(len(loader))

    for doc in documents:
        print(doc.page_content)
        content_id = client.add_documents("testing", doc.page_content)
        client.wait_for_extraction(content_id)




In [1]:
import dspy


In [84]:
lm = dspy.OllamaLocal(model="llama3",timeout_s = 180)


In [85]:
import dspy
from indexify import IndexifyClient
from indexify_dspy import retriever


In [86]:
index_name = "testing.embeddings.embedding"

In [87]:
indexify_client = IndexifyClient()
indexify_retriever_model = retriever.IndexifyRM(index_name,indexify_client)

dspy.settings.configure(lm=lm, rm=indexify_retriever_model)

In [95]:
indexify_client = IndexifyClient()
indexify_retriever_model = retriever.IndexifyRM(index_name,indexify_client)

dspy.settings.configure(lm=lm, rm=indexify_retriever_model)

In [ ]:
def get_context(question):
    retrieve = retriever.IndexifyRM(indexify_client)
    context = retrieve(question, index_name, k=3).passages

    return context

In [78]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="an explained answer")

In [92]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        
        context = get_context(question)
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)



In [93]:
uncompiled_rag = RAG()


In [94]:
my_question = "what is prediction rule"
response = uncompiled_rag(my_question).answer
print(response)

A prediction rule is a two-step process in machine learning used for making predictions. The first step is to "fit" or train a model to the data, where the model is a hypothesis or prediction rule y=h(x;θ) with parameters θ determined by the training data. The second step is to use the model to make predictions on new, un
